# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadGhGraphs import loadGhGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load test data

In [2]:
dataDir = "data/2D_Truss_v1.3/gh/"
testFile = os.path.join(dataDir, 'design_9_N_1000.csv')
allGraphsUnfiltered = loadGhGraphs(testFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.141483
std,1.574372
min,0.006253
25%,0.021285
50%,0.035028
75%,0.072772
max,48.701946


In [3]:
testData = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testData]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max().item()
source.describe()

,maxes
count,900.000000
mean,0.042389
std,0.030990
min,0.006253
25%,0.020052
50%,0.031303
75%,0.055108
max,0.140049


## 2. Load pre-train data

In [4]:
pretrainFiles = glob.glob(os.path.join(dataDir, '*1000.csv'))
pretrainFiles.remove(testFile)

allPretrainGraphs = []
for pretrainFile in pretrainFiles:
    designName = pretrainFile.split('/')[-1].split('_N')[0]
    print(f'loading {designName}')
    graphsUnfiltered = loadGhGraphs(pretrainFile, NUM_DV=5)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

loading design_7
loading design_6
loading design_8
loading design_5
loaded 3600 pretraining graphs


## 3. Initial training

In [5]:
saveDir = 'results/transferLrn_des9_01/'
epochs = 100
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(pretrainData, pretrainValData, 
                            epochs=epochs,
                            saveDir=saveDir+f'preTrain/gcn/')

ptrGcnCheckptFile = ptrGcn.checkptFile
plotHistory(history)

epoch: 0   trainLoss: 7.1136e-01   valLoss:1.1171e+00  time: 7.39e+00
epoch: 1   trainLoss: 3.7696e-01   valLoss:1.3573e+00  time: 7.10e+00
epoch: 2   trainLoss: 2.4571e-01   valLoss:3.6096e-01  time: 7.14e+00
epoch: 3   trainLoss: 2.0012e-01   valLoss:3.8529e-01  time: 7.11e+00
epoch: 4   trainLoss: 1.4104e-01   valLoss:3.6666e-01  time: 7.15e+00
epoch: 5   trainLoss: 1.1838e-01   valLoss:2.9482e-01  time: 7.16e+00
epoch: 6   trainLoss: 1.1851e-01   valLoss:4.0545e-01  time: 7.12e+00
epoch: 7   trainLoss: 1.2444e-01   valLoss:3.0170e-01  time: 7.14e+00
epoch: 8   trainLoss: 1.0371e-01   valLoss:3.1434e-01  time: 7.12e+00
epoch: 9   trainLoss: 9.7778e-02   valLoss:2.3398e-01  time: 7.17e+00
epoch: 10   trainLoss: 9.3861e-02   valLoss:1.9597e-01  time: 7.17e+00
epoch: 11   trainLoss: 7.7897e-02   valLoss:3.3217e-01  time: 7.15e+00
epoch: 12   trainLoss: 8.2173e-02   valLoss:1.3892e-01  time: 7.21e+00
epoch: 13   trainLoss: 7.9765e-02   valLoss:1.5254e-01  time: 7.14e+00
epoch: 14   trai

alt.Chart(...)

In [6]:
trainRes = ptrGcn.testModel(pretrainData)
testRes = ptrGcn.testModel(testData) # unseen topology
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000013,0.002384,0.097664,0.947526,NaN,NaN,NaN
test,0.000071,0.005430,0.181216,0.764785,0.92666,0.79292,0.482823


## 4. Transfer learning study

In [7]:
trainDataFiles = glob.glob(os.path.join(dataDir, 'design_9*.csv'))
trainDataFiles.remove(testFile)

allResults = []
for trainDataFile in trainDataFiles:
    trainDataUnfiltered = loadGhGraphs(trainDataFile, NUM_DV=5)
    trainData = filterbyDispValue(trainDataUnfiltered, maxDispCutoff)
    trainSize = len(trainData)
    print(f'loaded train set of size {trainSize}')
    
    
    ### fresh neural network ###
    gcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                         epochs=epochs, 
                         saveDir=saveDir+f'{trainSize:05}/gcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Fresh'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Fresh'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)

    
    ### transfer learning ###
    ptrGcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                             restartFile=ptrGcnCheckptFile,
                             epochs=epochs, 
                             saveDir=saveDir+f'{trainSize:05}/ptrGcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Transfer learning'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Transfer learning'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
    ### random forest ###
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                     saveDir=saveDir+f'{trainSize:05}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['Model'] = 'Random Forest'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['Model'] = 'Random Forest'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
pd.DataFrame(allResults)

loaded train set of size 48
epoch: 0   trainLoss: 1.0095e+00   valLoss:9.8591e-01  time: 4.83e-01
epoch: 1   trainLoss: 7.6828e-01   valLoss:9.7141e-01  time: 4.54e-01
epoch: 2   trainLoss: 6.0922e-01   valLoss:9.5443e-01  time: 4.48e-01
epoch: 3   trainLoss: 4.6051e-01   valLoss:9.3796e-01  time: 4.48e-01
epoch: 4   trainLoss: 3.5488e-01   valLoss:9.1689e-01  time: 4.49e-01
epoch: 5   trainLoss: 2.7095e-01   valLoss:8.9725e-01  time: 4.48e-01
epoch: 6   trainLoss: 2.3347e-01   valLoss:8.7435e-01  time: 4.40e-01
epoch: 7   trainLoss: 2.1106e-01   valLoss:8.3770e-01  time: 4.46e-01
epoch: 8   trainLoss: 1.9763e-01   valLoss:8.0761e-01  time: 4.44e-01
epoch: 9   trainLoss: 1.7392e-01   valLoss:7.9918e-01  time: 4.50e-01
epoch: 10   trainLoss: 1.5147e-01   valLoss:8.2972e-01  time: 4.52e-01
epoch: 11   trainLoss: 1.3430e-01   valLoss:9.2468e-01  time: 4.45e-01
epoch: 12   trainLoss: 1.1998e-01   valLoss:1.0653e+00  time: 4.42e-01
epoch: 13   trainLoss: 1.0428e-01   valLoss:1.1856e+00  tim

epoch: 15   trainLoss: 9.0670e-02   valLoss:1.7915e-01  time: 4.39e-01
epoch: 16   trainLoss: 8.4578e-02   valLoss:1.6388e-01  time: 4.44e-01
epoch: 17   trainLoss: 7.4507e-02   valLoss:1.8655e-01  time: 5.81e-01
epoch: 18   trainLoss: 7.0147e-02   valLoss:1.4120e-01  time: 4.36e-01
epoch: 19   trainLoss: 6.5836e-02   valLoss:1.0592e-01  time: 4.51e-01
epoch: 20   trainLoss: 5.9981e-02   valLoss:9.1253e-02  time: 4.59e-01
epoch: 21   trainLoss: 5.5810e-02   valLoss:8.2557e-02  time: 4.47e-01
epoch: 22   trainLoss: 5.0816e-02   valLoss:1.1018e-01  time: 4.48e-01
epoch: 23   trainLoss: 4.8908e-02   valLoss:9.8116e-02  time: 4.37e-01
epoch: 24   trainLoss: 4.4775e-02   valLoss:7.4243e-02  time: 4.52e-01
epoch: 25   trainLoss: 4.2807e-02   valLoss:6.2059e-02  time: 4.47e-01
epoch: 26   trainLoss: 3.9557e-02   valLoss:6.5402e-02  time: 4.35e-01
epoch: 27   trainLoss: 3.8201e-02   valLoss:6.6998e-02  time: 4.35e-01
epoch: 28   trainLoss: 3.7693e-02   valLoss:5.0902e-02  time: 4.41e-01
epoch:

epoch: 30   trainLoss: 3.2252e-02   valLoss:3.5199e-01  time: 1.82e-01
epoch: 31   trainLoss: 2.9247e-02   valLoss:3.4685e-01  time: 1.82e-01
epoch: 32   trainLoss: 2.7147e-02   valLoss:3.9485e-01  time: 1.82e-01
epoch: 33   trainLoss: 2.6721e-02   valLoss:3.2250e-01  time: 1.77e-01
epoch: 34   trainLoss: 3.0171e-02   valLoss:4.1609e-01  time: 1.82e-01
epoch: 35   trainLoss: 2.7271e-02   valLoss:3.6191e-01  time: 1.85e-01
epoch: 36   trainLoss: 2.1344e-02   valLoss:2.6813e-01  time: 1.78e-01
epoch: 37   trainLoss: 2.1967e-02   valLoss:2.9462e-01  time: 1.80e-01
epoch: 38   trainLoss: 1.6005e-02   valLoss:3.3150e-01  time: 1.78e-01
epoch: 39   trainLoss: 1.7734e-02   valLoss:2.9390e-01  time: 1.80e-01
epoch: 40   trainLoss: 1.3907e-02   valLoss:2.6013e-01  time: 1.79e-01
epoch: 41   trainLoss: 1.3640e-02   valLoss:2.6525e-01  time: 1.81e-01
epoch: 42   trainLoss: 1.2786e-02   valLoss:2.7836e-01  time: 1.84e-01
epoch: 43   trainLoss: 1.2403e-02   valLoss:2.5111e-01  time: 1.80e-01
epoch:

epoch: 45   trainLoss: 6.0596e-03   valLoss:1.0726e-02  time: 1.82e-01
epoch: 46   trainLoss: 5.8421e-03   valLoss:9.4335e-03  time: 1.81e-01
epoch: 47   trainLoss: 5.6577e-03   valLoss:1.4804e-02  time: 1.83e-01
epoch: 48   trainLoss: 5.7437e-03   valLoss:1.7036e-02  time: 1.82e-01
epoch: 49   trainLoss: 6.4574e-03   valLoss:2.4386e-02  time: 1.81e-01
epoch: 50   trainLoss: 8.4161e-03   valLoss:2.0284e-02  time: 1.86e-01
epoch: 51   trainLoss: 1.3765e-02   valLoss:2.8691e-02  time: 1.84e-01
epoch: 52   trainLoss: 1.4084e-02   valLoss:1.2370e-02  time: 1.82e-01
epoch: 53   trainLoss: 8.9679e-03   valLoss:1.6184e-02  time: 1.80e-01
epoch: 54   trainLoss: 5.3649e-03   valLoss:2.7126e-02  time: 1.81e-01
epoch: 55   trainLoss: 9.7058e-03   valLoss:1.7809e-02  time: 1.84e-01
epoch: 56   trainLoss: 6.9031e-03   valLoss:2.3858e-02  time: 1.83e-01
epoch: 57   trainLoss: 5.3149e-03   valLoss:2.8055e-02  time: 1.83e-01
epoch: 58   trainLoss: 7.9035e-03   valLoss:1.9596e-02  time: 1.84e-01
epoch:

epoch: 60   trainLoss: 3.8061e-02   valLoss:3.4328e-01  time: 4.03e+00
epoch: 61   trainLoss: 3.7284e-02   valLoss:3.4954e-01  time: 3.94e+00
epoch: 62   trainLoss: 3.6856e-02   valLoss:3.7524e-01  time: 3.93e+00
epoch: 63   trainLoss: 2.8998e-02   valLoss:4.6205e-01  time: 3.92e+00
epoch: 64   trainLoss: 3.7340e-02   valLoss:2.7493e-01  time: 3.92e+00
epoch: 65   trainLoss: 2.8318e-02   valLoss:3.4414e-01  time: 3.97e+00
epoch: 66   trainLoss: 2.8469e-02   valLoss:3.9531e-01  time: 4.06e+00
epoch: 67   trainLoss: 2.6688e-02   valLoss:1.9958e-01  time: 3.94e+00
epoch: 68   trainLoss: 2.7429e-02   valLoss:3.8636e-01  time: 3.94e+00
epoch: 69   trainLoss: 2.4074e-02   valLoss:2.7550e-01  time: 3.94e+00
epoch: 70   trainLoss: 3.1552e-02   valLoss:3.1639e-01  time: 4.01e+00
epoch: 71   trainLoss: 2.6527e-02   valLoss:3.3329e-01  time: 3.92e+00
epoch: 72   trainLoss: 2.2576e-02   valLoss:1.8574e-01  time: 3.97e+00
epoch: 73   trainLoss: 2.1580e-02   valLoss:4.1440e-01  time: 4.03e+00
epoch:

...

epoch: 50   trainLoss: 1.4924e-02   valLoss:4.0695e-02  time: 7.63e-01
epoch: 51   trainLoss: 1.2300e-02   valLoss:4.8715e-02  time: 7.71e-01
epoch: 52   trainLoss: 1.3013e-02   valLoss:4.8294e-02  time: 7.63e-01
epoch: 53   trainLoss: 1.1541e-02   valLoss:4.9524e-02  time: 7.75e-01
epoch: 54   trainLoss: 1.1339e-02   valLoss:5.6279e-02  time: 7.62e-01
epoch: 55   trainLoss: 1.0823e-02   valLoss:4.8141e-02  time: 7.70e-01
epoch: 56   trainLoss: 1.0040e-02   valLoss:5.5287e-02  time: 7.63e-01
epoch: 57   trainLoss: 1.0710e-02   valLoss:3.7747e-02  time: 7.65e-01
epoch: 58   trainLoss: 9.6575e-03   valLoss:5.1792e-02  time: 7.62e-01
epoch: 59   trainLoss: 1.0439e-02   valLoss:2.9104e-02  time: 7.70e-01
epoch: 60   trainLoss: 1.1780e-02   valLoss:4.3225e-02  time: 7.64e-01
epoch: 61   trainLoss: 1.4771e-02   valLoss:2.5177e-02  time: 7.76e-01
epoch: 62   trainLoss: 1.8794e-02   valLoss:2.2605e-02  time: 7.65e-01
epoch: 63   trainLoss: 1.4561e-02   valLoss:2.0733e-02  time: 7.61e-01
epoch:

epoch: 65   trainLoss: 3.7134e-02   valLoss:5.0719e-02  time: 7.68e+00
epoch: 66   trainLoss: 3.2988e-02   valLoss:6.1401e-02  time: 7.49e+00
epoch: 67   trainLoss: 3.1562e-02   valLoss:9.7060e-02  time: 7.68e+00
epoch: 68   trainLoss: 2.7845e-02   valLoss:4.2551e-02  time: 7.60e+00
epoch: 69   trainLoss: 3.6683e-02   valLoss:1.2656e-01  time: 7.62e+00
epoch: 70   trainLoss: 3.2849e-02   valLoss:7.3578e-02  time: 7.66e+00
epoch: 71   trainLoss: 3.5226e-02   valLoss:9.1978e-02  time: 7.64e+00
epoch: 72   trainLoss: 3.8125e-02   valLoss:8.6695e-02  time: 7.51e+00
epoch: 73   trainLoss: 3.5703e-02   valLoss:1.3566e-01  time: 7.57e+00
epoch: 74   trainLoss: 2.9719e-02   valLoss:6.1874e-02  time: 7.65e+00
epoch: 75   trainLoss: 2.9807e-02   valLoss:4.5844e-02  time: 7.69e+00
epoch: 76   trainLoss: 3.2279e-02   valLoss:2.9030e-02  time: 7.68e+00
epoch: 77   trainLoss: 3.7649e-02   valLoss:3.5388e-02  time: 7.70e+00
epoch: 78   trainLoss: 2.6476e-02   valLoss:6.1833e-02  time: 7.58e+00
epoch:

epoch: 80   trainLoss: 3.5046e-02   valLoss:6.3371e-02  time: 7.58e+00
epoch: 81   trainLoss: 3.8229e-02   valLoss:8.5149e-02  time: 7.59e+00
epoch: 82   trainLoss: 4.9869e-02   valLoss:8.4540e-02  time: 7.69e+00
epoch: 83   trainLoss: 4.2835e-02   valLoss:2.0279e-01  time: 7.54e+00
epoch: 84   trainLoss: 4.2697e-02   valLoss:6.2192e-02  time: 7.61e+00
epoch: 85   trainLoss: 3.0914e-02   valLoss:4.1289e-02  time: 7.53e+00
epoch: 86   trainLoss: 3.4322e-02   valLoss:7.3974e-02  time: 7.54e+00
epoch: 87   trainLoss: 3.4048e-02   valLoss:8.1519e-02  time: 7.55e+00
epoch: 88   trainLoss: 4.8824e-02   valLoss:1.0491e-01  time: 7.62e+00
epoch: 89   trainLoss: 4.1110e-02   valLoss:3.3855e-02  time: 7.58e+00
epoch: 90   trainLoss: 3.4265e-02   valLoss:6.9165e-02  time: 7.63e+00
epoch: 91   trainLoss: 4.5176e-02   valLoss:5.2259e-02  time: 7.56e+00
epoch: 92   trainLoss: 2.9993e-02   valLoss:3.4398e-02  time: 7.59e+00
epoch: 93   trainLoss: 3.3490e-02   valLoss:7.1520e-02  time: 7.54e+00
epoch:

epoch: 97   trainLoss: 3.7367e-04   valLoss:6.9584e-02  time: 8.59e-02
epoch: 98   trainLoss: 2.3925e-04   valLoss:6.6789e-02  time: 8.53e-02
epoch: 99   trainLoss: 2.9850e-04   valLoss:6.8530e-02  time: 8.82e-02
loading checkpoint 98
loading restart file
epoch: 0   trainLoss: 3.2338e-01   valLoss:2.1569e+00  time: 8.60e-02
epoch: 1   trainLoss: 2.8858e-01   valLoss:2.0624e+00  time: 8.64e-02
epoch: 2   trainLoss: 1.7672e-01   valLoss:5.2127e-01  time: 8.62e-02
epoch: 3   trainLoss: 1.5479e-01   valLoss:5.4814e-01  time: 8.96e-02
epoch: 4   trainLoss: 1.3949e-01   valLoss:6.0858e-01  time: 8.78e-02
epoch: 5   trainLoss: 7.9400e-02   valLoss:3.6663e-01  time: 8.72e-02
epoch: 6   trainLoss: 7.1359e-02   valLoss:2.8271e-01  time: 8.80e-02
epoch: 7   trainLoss: 6.2293e-02   valLoss:1.8289e-01  time: 8.62e-02
epoch: 8   trainLoss: 5.1838e-02   valLoss:1.7411e-01  time: 8.60e-02
epoch: 9   trainLoss: 4.7335e-02   valLoss:1.9463e-01  time: 8.69e-02
epoch: 10   trainLoss: 4.1310e-02   valLoss:

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,0.000087,0.006301,0.196867,0.750303,0.900059,0.724378,-1.629229,Fresh,Train,48
1,0.000155,0.008055,0.249037,0.368699,0.687472,0.490062,-1.091653,Fresh,Test,48
2,0.000007,0.001848,0.070574,0.975361,0.996057,0.959733,0.572509,Transfer learning,Train,48
3,0.000061,0.004401,0.142920,0.725684,0.907137,0.790991,0.175821,Transfer learning,Test,48
4,0.000022,0.002893,0.091106,0.925249,0.981562,0.949793,0.874099,Random Forest,Train,48
5,0.000114,0.006645,0.207026,0.519149,0.756080,0.675091,0.542115,Random Forest,Test,48
6,0.000038,0.004346,0.160157,0.931194,0.960480,0.788970,-1.088449,Fresh,Train,18
7,0.000213,0.009103,0.284163,0.368963,0.628796,0.219805,-3.422486,Fresh,Test,18
8,0.000003,0.001264,0.054617,0.996814,0.997626,0.966141,0.365412,Transfer learning,Train,18
9,0.000101,0.005767,0.177836,0.601090,0.834832,0.666008,-0.336928,Transfer learning,Test,18


In [8]:
# df = pd.DataFrame(allResults)
df = pd.read_csv('results/transferLrn_des9_01/testResults.csv')
df

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,0.000087,0.006301,0.196867,0.750303,0.900059,0.724378,-1.629229,Fresh,Train,48
1,0.000155,0.008055,0.249037,0.368699,0.687472,0.490062,-1.091653,Fresh,Test,48
2,0.000007,0.001848,0.070574,0.975361,0.996057,0.959733,0.572509,Transfer learning,Train,48
3,0.000061,0.004401,0.142920,0.725684,0.907137,0.790991,0.175821,Transfer learning,Test,48
4,0.000022,0.002893,0.091106,0.925249,0.981562,0.949793,0.874099,Random Forest,Train,48
5,0.000114,0.006645,0.207026,0.519149,0.756080,0.675091,0.542115,Random Forest,Test,48
6,0.000038,0.004346,0.160157,0.931194,0.960480,0.788970,-1.088449,Fresh,Train,18
7,0.000213,0.009103,0.284163,0.368963,0.628796,0.219805,-3.422486,Fresh,Test,18
8,0.000003,0.001264,0.054617,0.996814,0.997626,0.966141,0.365412,Transfer learning,Train,18
9,0.000101,0.005767,0.177836,0.601090,0.834832,0.666008,-0.336928,Transfer learning,Test,18


In [9]:
df = df.replace('Fresh', 'GCN')
df = df.replace('Transfer learning', 'GCN with transfer learning')

In [10]:
alt.Chart(df[df.Set=='Test']).mark_circle().encode(
    x=alt.X('Train Size:Q', scale=alt.Scale(type='log')),
    y=alt.Y('mse:Q', title='MSE', axis=alt.Axis(format='.1e')),
    color='Model',
    tooltip=['Model', 'mse']
).properties(width=400, height=200, title='Design Group 9')

alt.Chart(...)